## flixpatrol 사이트에서 데이터 수집
- https://flixpatrol.com/top10/netflix/world/2021/full/#netflix-1

### TOP Movies on Netflix in 2021

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
def flixpatrol_top_movies(pick_css):
    url = "https://flixpatrol.com/top10/netflix/world/2021/full/#netflix-1"
    
    response = requests.get(url)
    dom = BeautifulSoup(response.text, "html.parser")
    elements = dom.select(pick_css)
    
    datas = []
    for ele in elements:
        # 데이터 전처리
        # 1. isNetflixOriginal
        try:
            ele.select("td")[2].select_one("span").get("title")
            netflix_original = "Netflix original title"
        except:
            netflix_original = "The general title"
        
        # 2. change
        is_change = ele.select("td")[4].text.strip()
        if is_change == '–':
            is_change = "None"
            
        # 데이터 수집
        datas.append({
            "rank" : int(ele.select("td")[0].text.split(".")[0]),
            "link" : "https://flixpatrol.com" + ele.select("td")[2].select_one("a").get("href"),
            "rankChange" : ele.select("td")[1].text.strip(),
            "title" : ele.select("td")[2].text.strip(),
            "isNetflixOriginal" : netflix_original,
            "points" : int(ele.select("td")[3].text.strip().replace(',', '')),
            "change" : is_change,
            "countriesCnt" : int(ele.select("td")[5].text.strip().replace(',', '')),
            "totalCountries" : int(ele.select("td")[6].text.strip().replace(',', '')),
            "days" : int(ele.select("td")[7].text.strip().replace(',', '')),
            "totalDays" : int(ele.select("td")[8].text.strip().replace(',', '')),
            "total" : int(ele.select("td")[9].text.strip().replace(',', ''))
        })
            
    return pd.DataFrame(datas)

In [4]:
pick_css = "#netflix-1 .table-group"
movies = flixpatrol_top_movies(pick_css)
movies.tail(10)

,rank,link,rankChange,title,isNetflixOriginal,points,change,countriesCnt,totalCountries,days,totalDays,total
166,167,https://flixpatrol.com/title/room/,n/a,Room,The general title,1922,None,22,87,20,96,1922
167,168,https://flixpatrol.com/title/i-see-you/,n/a,I See You,The general title,1889,None,18,105,15,126,1889
168,169,https://flixpatrol.com/title/2012/,n/a,2012,The general title,1886,None,31,61,51,37,1886
169,170,https://flixpatrol.com/title/wanted/,n/a,Wanted,The general title,1878,None,32,59,20,94,1878
170,171,https://flixpatrol.com/title/just-say-yes/,n/a,Just Say Yes,Netflix original title,1878,None,43,44,16,117,1878
171,172,https://flixpatrol.com/title/scoob/,n/a,Scoob!,The general title,202,None,1,202,28,7,202
172,173,https://flixpatrol.com/title/the-son-of-bigfoot/,n/a,The Son of Bigfoot,The general title,192,None,7,27,8,24,192
173,174,https://flixpatrol.com/title/ava-2020/,n/a,Ava,The general title,186,None,4,47,29,6,186
174,175,https://flixpatrol.com/title/bombshell/,n/a,Bombshell,The general title,181,None,2,91,14,13,181
175,176,https://flixpatrol.com/title/man-on-a-ledge/,n/a,Man on a Ledge,The general title,174,None,4,44,13,13,174


In [5]:
def movies_infro(movies):
    debug = 0
    all_data = []
    for tit, url in zip(movies["title"], movies["link"]):
        # 데이터 만들기
        try:
            rep = find_moive_details_info(url)

            # 변수 만들기
            rep_value = list(rep.values())
            rep_value.append(tit)
            all_data.append(rep_value)
        except:
            print("error : ", tit)

    rep_key = list(rep.keys())
    rep_key.append("title")
    
    movies_detail = pd.DataFrame(all_data, columns = rep_key)
        
    return movies_detail

def find_moive_details_info(url):
    response = requests.get(url)
    dom = BeautifulSoup(response.text, "html.parser")

    # 특징 있는 class 이름 아님
    elements = dom.select(".content")
    ele = elements[1]
    ele.select("span")
    
    #데이터 처리
    find_info_idx = []
    result = {}
    for idx, data in enumerate(ele.select("span")):
        if "mx-1 text-gray-600 select-none" in str(data):
            find_info_idx.append(idx + 1)

    movie_info_details = []
    for idx in find_info_idx:
        movie_info_details.append(ele.select("span")[idx].text)


    # 만들어진 나라
    result["MovieProductionCountry"] = movie_info_details[0]    

    # 개봉 월/ 일/ 년
    opening_date = movie_info_details[1].split("/")
    result["year"] = opening_date[-1]
    result["month"] = opening_date[0]
    result["day"] = opening_date[1]

    # 영화 분류
    result['genre'] = movie_info_details[-2]

    # 영화 분류
    result['sub_genre'] = movie_info_details[-1]

    return result
    
movies_add = movies_infro(movies)
print("end")

error :  The Magnificent Seven
error :  Ride Along 2
error :  Dracula Untold
error :  Ted 2
error :  Men in Black: International
error :  The Adventures of Sharkboy and Lavagirl
error :  Rurouni Kenshin Part I: Origins
error :  Sniper: Ultimate Kill
error :  Angel Has Fallen
error :  The Son of Bigfoot
end


In [6]:
movies_add.head(100)

,MovieProductionCountry,year,month,day,genre,sub_genre,title
0,United States,2021,11,12,Action,Treasure hunt,Red Notice
1,United States,2020,12,25,Family,,We Can Be Heroes
2,United States,2021,05,14,Zombies,,Army of the Dead
3,China,2021,06,11,Animated,Family,Wish Dragon
4,Germany,2021,10,29,Heist,,Army of Thieves
...,...,...,...,...,...,...,...
95,United States,2021,07,09,Slasher,,Fear Street: 1978
96,South Africa,2021,05,14,Thriller,Disapperance,I Am All Girls
97,United States,2021,03,26,Comedy,Prank Show,Bad Trip
98,Brazil,2021,01,15,Comedy,Rom Com,Double Dad


In [14]:
# 행 결합
# final_movies = pd.merge(left = movies , right = movies_add, how = "left", on = "title")
final_movies = pd.merge(left = movies , right = movies_add, how = "inner", on = "title")
final_movies.tail(2)

,rank,link,rankChange,title,isNetflixOriginal,points,change,countriesCnt,totalCountries,days,totalDays,total,MovieProductionCountry,year,month,day,genre,sub_genre
164,175,https://flixpatrol.com/title/bombshell/,n/a,Bombshell,The general title,181,None,2,91,14,13,181,United States,2019,12,13,Drama,Journalists
165,176,https://flixpatrol.com/title/man-on-a-ledge/,n/a,Man on a Ledge,The general title,174,None,4,44,13,13,174,United States,2012,01,27,Thriller,Heist


In [24]:
final_movies.to_csv("netflix_2021.csv", mode='w')

In [28]:
df = pd.read_csv("netflix_2021.csv", index_col=0)
df.tail(2)

,rank,link,rankChange,title,isNetflixOriginal,points,change,countriesCnt,totalCountries,days,totalDays,total,MovieProductionCountry,year,month,day,genre,sub_genre
164,175,https://flixpatrol.com/title/bombshell/,NaN,Bombshell,The general title,181,None,2,91,14,13,181,United States,2019,12,13,Drama,Journalists
165,176,https://flixpatrol.com/title/man-on-a-ledge/,NaN,Man on a Ledge,The general title,174,None,4,44,13,13,174,United States,2012,1,27,Thriller,Heist


### TOP TV Shows on Netflix in 2021

In [13]:
pick_css = "#netflix-2 .table-group"
tv_shows = flixpatrol_top_movies(pick_css)
tv_shows.tail(10)

,rank,link,rankChange,title,isNetflixOriginal,points,change,countriesCnt,totalCountries,days,totalDays,total
142,143,https://flixpatrol.com/title/rupauls-drag-race/,n/a,RuPaul's Drag Race,The general title,2005,None,25,80,74,27,2005
143,144,https://flixpatrol.com/title/castlevania/,n/a,Castlevania,Netflix original title,1957,None,67,29,20,98,1957
144,145,https://flixpatrol.com/title/workin-moms/,n/a,Workin' Moms,Netflix original title,1924,None,47,41,18,107,1924
145,146,https://flixpatrol.com/title/attenboroughs-lif...,n/a,Attenborough's Life in Colour,Netflix original title,1908,None,51,37,13,147,1908
146,147,https://flixpatrol.com/title/crash-landing-on-...,n/a,Crash Landing on You,Netflix original title,1891,None,9,210,180,11,1891
147,148,https://flixpatrol.com/title/fatma-2021/,n/a,Fatma,Netflix original title,1680,None,44,38,47,36,1680
148,149,https://flixpatrol.com/title/sweet-home-2020/,n/a,Sweet Home,Netflix original title,1656,None,22,75,50,33,1656
149,150,https://flixpatrol.com/title/outlander-2014/,n/a,Outlander,The general title,1642,None,22,75,104,16,1642
150,151,https://flixpatrol.com/title/beauty-and-the-ba...,n/a,Beauty and the Baker,The general title,1638,None,18,91,28,58,1638
151,152,https://flixpatrol.com/title/zero-chill/,n/a,Zero Chill,Netflix original title,1631,None,49,33,20,82,1631


In [29]:
tv_shows.to_csv("netflix_tv_show_2021.csv", mode='w')

In [30]:
df = pd.read_csv("netflix_tv_show_2021.csv", index_col=0)
df.tail(2)

,rank,link,rankChange,title,isNetflixOriginal,points,change,countriesCnt,totalCountries,days,totalDays,total
150,151,https://flixpatrol.com/title/beauty-and-the-ba...,NaN,Beauty and the Baker,The general title,1638,None,18,91,28,58,1638
151,152,https://flixpatrol.com/title/zero-chill/,NaN,Zero Chill,Netflix original title,1631,None,49,33,20,82,1631
